In [4]:
#!pip install surprise
#!pip install import-ipynb
import os
import csv
import pandas as pd
import numpy as np
import heapq
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
import pickle
from surprise import accuracy
from collections import defaultdict
from surprise import KNNBasic
from collections import defaultdict
from operator import itemgetter
import NetflixLoadData as NetflixLoadData
#sns.set_style("darkgrid")

## Get data

In [5]:
use_pickle_file = True
max_n = 5000000 #how many rows we want from data_ratings and data_rating_plus_movie_title
reader = Reader(line_format='user item rating', rating_scale=(1, 5))

in these txt files the movie_id's have been placed amongst the customer_id's, which means that we have to split that column into two, we do this by taking all ratings that are equal to NaN 
( meaning that we are now refering to a movie and not a customer ) and from there we split it into two columns.

In [6]:
data_movies, tmp_data_rating, tmp_data_rating_plus_movie_title, data_movies_categorized = NetflixLoadData.get_dataframes(use_pickle=use_pickle_file)

/home/ragnar/netflix-recommendation/NetflixLoadData.py:82: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  data_movies_categorize_split[1] = data_movies_categorize_split[1].str.replace(r')', '') #removing ) at the end of movie_year
/home/ragnar/netflix-recommendation/NetflixLoadData.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_movies_categorize_cleaned["movie_year"] = data_movies_categorize_cleaned["movie_year"].astype(float)


In [7]:
data_rating, data_rating_plus_movie_title = NetflixLoadData.get_sample_of_data(tmp_data_rating, tmp_data_rating_plus_movie_title, max_n=max_n)

## Helper functions

In [8]:
def save_to_pickle(name, df):
    path_name = "pickle/"+name+".pickle"
    pickle_file = open(path_name,"wb")
    pickle.dump(df, pickle_file)
    pickle_file.close()

def load_pickle(name):
    path_name = "pickle/"+name+".pickle"
    return_input = open(path_name, "rb")
    return pickle.load(return_input)

## Finding information about the data before doing anything major

In [9]:
import MovieCustomerInformation as information
max_rating = 4
min_rating = 4

### Details about all customers/movies

In [10]:
# get the average movie rating for all customers
# used to determine if this user typically gives bad or good reviews
# and then we can see if he really hates or loves a movie
all_customers_average_ratings =  information.all_average_ratings(df=data_rating, type='customer_id')
print(all_customers_average_ratings)

       customer_id rating       avg_rating
                      sum count           
0                6   95.0    28   3.392857
1                7  159.0    37   4.297297
2               10   40.0    12   3.333333
3               25    7.0     2   3.500000
4               33    9.0     3   3.000000
...            ...    ...   ...        ...
404473     2649401   11.0     3   3.666667
404474     2649404   12.0     3   4.000000
404475     2649409   15.0     4   3.750000
404476     2649426   65.0    15   4.333333
404477     2649429   56.0    14   4.000000

[404478 rows x 4 columns]


In [11]:
all_movies_average_rating = information.all_average_ratings(df=data_rating, type='movie_id')
print(all_movies_average_rating)

    movie_id   rating        avg_rating
                  sum  count           
0          1   2051.0    547   3.749543
1          2    516.0    145   3.558621
2          3   7326.0   2012   3.641153
3          4    389.0    142   2.739437
4          5   4468.0   1140   3.919298
..       ...      ...    ...        ...
991      992  24433.0   6210   3.934461
992      993  10185.0   3337   3.052143
993      994  42674.0  13253   3.219950
994      995    179.0    104   1.721154
995      996  17489.0   4323   4.045570

[996 rows x 4 columns]


In [12]:
information.get_avg_rating_less_than(df=all_customers_average_ratings , max_rating=max_rating)

       customer_id rating       avg_rating
                      sum count           
0                6   95.0    28   3.392857
2               10   40.0    12   3.333333
3               25    7.0     2   3.500000
4               33    9.0     3   3.000000
5               42   22.0     6   3.666667
...            ...    ...   ...        ...
404468     2649370   49.0    13   3.769231
404471     2649378   44.0    14   3.142857
404472     2649388   40.0    13   3.076923
404473     2649401   11.0     3   3.666667
404475     2649409   15.0     4   3.750000

[248957 rows x 4 columns]


In [13]:
information.get_avg_rating_higher_than(df=all_customers_average_ratings, min_rating=min_rating)

       customer_id rating       avg_rating
                      sum count           
1                7  159.0    37   4.297297
6               59   50.0    12   4.166667
13             126    5.0     1   5.000000
16             134  160.0    34   4.705882
19             164   33.0     8   4.125000
...            ...    ...   ...        ...
404460     2649311   60.0    14   4.285714
404464     2649331   46.0    11   4.181818
404469     2649375   48.0    11   4.363636
404470     2649376   36.0     8   4.500000
404476     2649426   65.0    15   4.333333

[102021 rows x 4 columns]


In [14]:
information.get_avg_rating_less_than(df=all_movies_average_rating, max_rating=max_rating)

    movie_id   rating        avg_rating
                  sum  count           
0          1   2051.0    547   3.749543
1          2    516.0    145   3.558621
2          3   7326.0   2012   3.641153
3          4    389.0    142   2.739437
4          5   4468.0   1140   3.919298
..       ...      ...    ...        ...
990      991   1312.0    359   3.654596
991      992  24433.0   6210   3.934461
992      993  10185.0   3337   3.052143
993      994  42674.0  13253   3.219950
994      995    179.0    104   1.721154

[939 rows x 4 columns]


In [15]:
information.get_avg_rating_higher_than(df=all_movies_average_rating, min_rating=min_rating)

    movie_id    rating         avg_rating
                   sum   count           
12        13     569.0     125   4.552000
31        32    7549.0    1854   4.071737
32        33   28722.0    6890   4.168650
67        68    9180.0    2216   4.142599
75        76   12083.0    2954   4.090386
84        85    1131.0     266   4.251880
105      106    3852.0     940   4.097872
134      135    1589.0     384   4.138021
166      167   37448.0    9151   4.092230
174      175  362148.0   90450   4.003847
208      209   18195.0    4373   4.160759
214      215   23632.0    5817   4.062575
222      223  101595.0   24125   4.211192
240      241  179297.0   43122   4.157901
252      253   20543.0    5050   4.067921
269      270  156266.0   36431   4.289369
315      316   10841.0    2520   4.301984
324      325    4703.0    1121   4.195361
344      345   17165.0    4175   4.111377
358      359   19027.0    4580   4.154367
420      421    3991.0     987   4.043566
429      430    6258.0    1546   4

In [16]:
count_movies_rated_by_customer = information.all_get_rated_count(df=data_rating, type="customer_id")
count_movies_rated_by_customer

,customer_id,movie_id
0,6,28
1,7,37
2,10,12
3,25,2
4,33,3
...,...,...
404473,2649401,3
404474,2649404,3
404475,2649409,4
404476,2649426,15


In [17]:
# def generate_random_user_ids(from_n, to_n):
#     ids = np.unique(data_rating['customer_id'])[from_n:to_n]
#     print("random user id used while in development\n")
#     return ids
# generate_random_user_ids(1000, 1500)

### Details about a specific customer

In [18]:
# ids I like -> 28812, 56514, 56520, 1488844
customer_id_use = 6 #84100

In [19]:
#all_custumers_id_rows(customer_id=customer_id_use)
## same as line above but including movie_year and movie_title

#all_customers_id_plus_movie_title_rows(customer_id=customer_id_use)
information.all_id_rows(df=data_rating_plus_movie_title, type="customer_id", item_id=customer_id_use)

,movie_id,customer_id,rating,movie_year,movie_title
187297,30,6,3.0,2003.0,Something's Gotta Give
539827,157,6,3.0,2004.0,Laird: White Knuckle Extreme
576723,173,6,4.0,1968.0,The Devil's Brigade
649632,175,6,5.0,1992.0,Reservoir Dogs
795442,191,6,2.0,2003.0,X2: X-Men United
881625,197,6,3.0,2004.0,Taking Lives
1053784,241,6,3.0,1959.0,North by Northwest
1368432,295,6,4.0,1995.0,Ace Ventura: When Nature Calls
1438379,299,6,3.0,2001.0,Bridget Jones's Diary
1723378,329,6,4.0,1999.0,Dogma


In [20]:
#get_users_avg_rating(customer_id=customer_id_use)
information.get_item_avg_rating(df=all_customers_average_ratings, type='customer_id', item_id=customer_id_use)

customer_id rating       avg_rating
                 sum count           
0           6   95.0    28   3.392857

In [21]:
#display_movies_customer_rated_higher_than(customer_id=customer_id_use, min_rating=4)
information.display_movies_customer_rated_higher_than(df=data_rating_plus_movie_title, customer_id=customer_id_use, min_rating=min_rating)

                                   movie_title  rating
movie_id                                              
173                        The Devil's Brigade     4.0
175                             Reservoir Dogs     5.0
295             Ace Ventura: When Nature Calls     4.0
329                                      Dogma     4.0
457                          Kill Bill: Vol. 2     5.0
528       The Hitchhiker's Guide to the Galaxy     4.0
564                                 Predator 2     4.0
886                                        Ray     5.0
985                                  The Mummy     4.0
average rating 0    3.392857
Name: avg_rating, dtype: float64


In [22]:
information.display_movies_customer_rated_lower_than(df= data_rating_plus_movie_title,customer_id=customer_id_use, max_rating=max_rating)

                                                movie_title  rating
movie_id                                                           
30                                   Something's Gotta Give     3.0
157                            Laird: White Knuckle Extreme     3.0
191                                        X2: X-Men United     2.0
197                                            Taking Lives     3.0
241                                      North by Northwest     3.0
299                                   Bridget Jones's Diary     3.0
361               The Phantom of the Opera: Special Edition     3.0
445                                         Michael Collins     3.0
468                                 The Matrix: Revolutions     3.0
494       Monty Python's The Meaning of Life: Special Ed...     3.0
501                                Three Days of the Condor     3.0
580                               THX 1138: Special Edition     3.0
658                           Robin Hood: Prince

In [23]:
information.get_users_loved_hated_movies(df=data_rating_plus_movie_title, customer_id=customer_id_use, minmax_rating=4)

User 6 loved these movies
The Devil's Brigade
Reservoir Dogs
Ace Ventura: When Nature Calls
Dogma
Kill Bill: Vol. 2
The Hitchhiker's Guide to the Galaxy
Predator 2
Ray
The Mummy

and disliked these movies
Something's Gotta Give
Laird: White Knuckle Extreme
X2: X-Men United
Taking Lives
North by Northwest
Bridget Jones's Diary
The Phantom of the Opera: Special Edition
Michael Collins
The Matrix: Revolutions
Monty Python's The Meaning of Life: Special Edition
Three Days of the Condor
THX 1138: Special Edition
Robin Hood: Prince of Thieves
Major League
Mobsters
Curly Sue
Clerks
Bill Cosby: Himself
Seven Samurai


## Recommendations with collaborative filtering

#### functions

In [24]:
from surprise import dataset, KNNBaseline, accuracy
from surprise.model_selection import train_test_split, LeaveOneOut

def get_drop_list(type="movie_id"):
    #>IF MOVIE:  movie_id, movie rating count, movie rating mean
    #>IF CUSTOMER: customer_id, custumer rating count, customer rating mean
    df_count_mean_summary = data_rating.groupby(type)['rating'].agg(['count', 'mean'])
    df_count_mean_summary.index = df_count_mean_summary.index.map(int)
    #>IF MOVIE: returns 1799.0 as a benchmark number
    #IF CUSTOMER: returns 52.0 as a benchmark number
    benchmark = round(df_count_mean_summary['count'].quantile(0.7),0)
    # drop all rows below benchmark
    df_drop_list = df_count_mean_summary[df_count_mean_summary['count'] < benchmark]
    # return all indexes to drop
    return df_drop_list

def get_customer_recommendations(customer_id, predictor):
    #> returns movie_id, movie_year, movie_title
    chosen_customer_pred = data_movies.copy()
    # fails if movie_id is the index so we have to reset the index back to normal (0-N)
    chosen_customer_pred = chosen_customer_pred.reset_index()
    # makes sure that we only pick movies that are not in the movie dropped list
    chosen_customer_pred = chosen_customer_pred[~chosen_customer_pred['movie_id'].isin(df_movie_drop_list)]
    # make prediction for customer with id = <customer_id> and put it into 'estimated_score'
    chosen_customer_pred['estimated_score'] = chosen_customer_pred['movie_id'].apply(lambda x: predictor.predict(customer_id, x).est)
    # sort by 'estimated score'
    chosen_customer_pred = chosen_customer_pred.sort_values('estimated_score', ascending=False).set_index('movie_id')
    return chosen_customer_pred

# print out movies/tv show user has previously rated
def display_rated_content(customer_id=customer_id_use, number_to_show=20):
    print("Movies/TV Shows rated by customer", customer_id)
    #df = all_customers_id_plus_movie_title_rows(customer_id=customer_id)
    df = information.all_id_rows(df=data_rating_plus_movie_title, type='customer_id', item_id=customer_id)
    df = df[['movie_title', 'rating']].sort_values('rating', ascending=False)
    print(df[0:number_to_show].set_index('movie_title'))

def display_customers_recommendations(customer_id=customer_id_use, df=[], number_to_show=20):
    print("Movies/TV Shows recommended to customer")
    tmp_df = df[['movie_title', 'estimated_score']][0:number_to_show]
    tmp_df = tmp_df.set_index('movie_title')
    print(tmp_df)

def display_recommendation(customer_id, number_to_show, predictor):
    chosen_customer_pred = get_customer_recommendations(customer_id=customer_id, predictor=predictor)
    display_customers_recommendations(customer_id=customer_id, df=chosen_customer_pred, number_to_show=number_to_show)

def print_evaluation_accuracy(prediction):
    print("\nEvaluating accuracy of model...")
    print("RMSE: ", accuracy.rmse(prediction, verbose=False))
    print("MAE: ", accuracy.mae(prediction, verbose=False))

#### create recommenders

In [25]:
## creating a list of the movies that we don't wanna include
df_movie_drop_list = get_drop_list(type="movie_id")
#df_customer_drop_list = get_summary(type="customer_id")

In [26]:
class convert_to_raw_ratings(dataset.DatasetAutoFolds):
    def __init__(self, df, reader):
        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in zip(df['customer_id'], df['movie_id'], df['rating'])]
        self.reader=reader

raw_ratings = convert_to_raw_ratings(data_rating, reader)
print("\nBuilding recommendation model...")
trainSet, testSet = train_test_split(raw_ratings, test_size=.25, random_state=1)

algo = SVD(random_state=10)
algo.fit(trainSet)
algo_predictions = algo.test(testSet)


Building recommendation model...


In [49]:
#save_to_pickle(name="algo_svd", df=algo)
#load_pickle("algo_svd")

In [27]:
svd = SVD(random_state=10)
data_new = Dataset.load_from_df(data_rating[['customer_id', 'movie_id', 'rating']], reader)
trainset = data_new.build_full_trainset()
svd.fit(trainset)
svd_predictions = svd.test(testSet)


In [50]:
#save_to_pickle(name="svd", df=svd)
#load_pickle("svd")

### display recommendations

In [28]:
number_to_show = 10

#### customer = 79724

In [29]:
display_rated_content(customer_id=79724, number_to_show=number_to_show)

Movies/TV Shows rated by customer 79724
                            rating
movie_title                       
Dogma                          5.0
American Beauty                5.0
What the #$*! Do We Know!?     4.0
Reservoir Dogs                 4.0
Ed Wood                        3.0
Pay It Forward                 3.0
Igby Goes Down                 3.0
Jaws                           3.0
Elephant                       2.0
Logan's Run                    2.0


In [30]:
display_recommendation(customer_id=79724, number_to_show=number_to_show, predictor=svd)

Movies/TV Shows recommended to customer
                                        estimated_score
movie_title                                            
Foyle's War: Set 2                             4.727764
Monarch of the Glen: Series 2                  4.621920
La Femme Nikita: Season 3                      4.587257
Invader Zim                                    4.494654
Ghosts of Rwanda: Frontline                    4.469619
Reno 911: Season 2                             4.446906
Yu Yu Hakusho                                  4.423891
A Touch of Frost: Seasons 7 & 8                4.415173
Homicide: Life on the Street: Season 7         4.402799
Mary Poppins: Bonus Material                   4.363708


In [31]:
display_recommendation(customer_id=79724, number_to_show=number_to_show, predictor=algo)

Movies/TV Shows recommended to customer
                                               estimated_score
movie_title                                                   
Xena: Warrior Princess: Season 3                      4.500141
Dogma                                                 4.461506
Yu Yu Hakusho                                         4.450138
The Tick: The Entire Series                           4.433872
GTO: Great Teacher Onizuka: Set 2                     4.373271
Viva La Bam: Season 1                                 4.370851
Absolutely Fabulous: Series 5                         4.351257
Sex and the City: Season 4                            4.346610
Unprecedented: The 2000 Presidential Election         4.318533
Clerks                                                4.314401


#### customer = customer_id_use

In [32]:
display_rated_content(customer_id=customer_id_use, number_to_show=number_to_show)

Movies/TV Shows rated by customer 6
                                      rating
movie_title                                 
Kill Bill: Vol. 2                        5.0
Reservoir Dogs                           5.0
Ray                                      5.0
The Mummy                                4.0
Ace Ventura: When Nature Calls           4.0
Dogma                                    4.0
Predator 2                               4.0
The Hitchhiker's Guide to the Galaxy     4.0
The Devil's Brigade                      4.0
Bill Cosby: Himself                      3.0


In [33]:
display_recommendation(customer_id=customer_id_use, number_to_show=number_to_show, predictor=svd)

Movies/TV Shows recommended to customer
                                                    estimated_score
movie_title                                                        
La Femme Nikita: Season 3                                  4.553385
Reno 911: Season 2                                         4.314836
Stevie Ray Vaughan and Double Trouble: Live at ...         4.306074
Baby Einstein: Baby Da Vinci                               4.235643
The Chorus                                                 4.232877
Ghosts of Rwanda: Frontline                                4.225248
Blue Planet: IMAX                                          4.220234
A Touch of Frost: Seasons 7 & 8                            4.219744
Foyle's War: Set 2                                         4.198104
Christmas with The Simpsons                                4.197200


In [34]:
display_recommendation(customer_id=customer_id_use, number_to_show=number_to_show, predictor=algo)
# recommends for example: Reservoir Dogs, which the user has already rated

Movies/TV Shows recommended to customer
                                                    estimated_score
movie_title                                                        
Foyle's War: Set 2                                         4.065095
That '70s Show: Season 1                                   4.032636
Princess Mononoke                                          4.025240
Highlander: Season 4                                       4.023670
Reno 911: Season 2                                         3.980902
Ghosts of Rwanda: Frontline                                3.973723
Baby Einstein: Baby Da Vinci                               3.962280
Lord of the Rings: The Return of the King: Exte...         3.945479
Sharpe 4: Sharpe's Enemy                                   3.939068
George Lopez: Why You Crying?                              3.920651


### print accuracy

In [35]:
print_evaluation_accuracy(svd_predictions)


Evaluating accuracy of model...
RMSE:  0.7642118848136733
MAE:  0.5967149757624542


In [36]:
print_evaluation_accuracy(algo_predictions)


Evaluating accuracy of model...
RMSE:  0.9602373221265598
MAE:  0.7530683221520933


## other

In [37]:

dataset = Dataset.load_from_df(data_rating[['customer_id', 'movie_id', 'rating']], reader)
trainset = dataset.build_full_trainset()

In [38]:
tmp_data_movies = data_movies[['movie_id', 'movie_title']]
tmp_data_movies = tmp_data_movies.set_index('movie_id').T
tmp_data_movies = tmp_data_movies.to_dict('list')
tmp_data_movies = {k: str(v[0]) for k,v in tmp_data_movies.items()}

In [39]:
similarity_matrix = KNNBasic(sim_options={'name': 'cosine', 'user_based': False}).fit(trainset).compute_similarities()

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [40]:
# generate_random_user_ids(1000, 1500)

In [41]:
# for i in generate_random_user_ids(1000, 1500):
#     if (len(all_custumers_id_rows(i)) > 3):
#         print(i)

In [42]:
test_subject = customer_id_use#16272
k = 10

In [43]:
test_subject_iid = trainset.to_inner_uid(test_subject)

test_subject_ratings = trainset.ur[test_subject_iid]
k_neighbours = heapq.nlargest(k, test_subject_ratings, key=lambda t: t[1])

In [44]:
candidates = defaultdict(float)

for itemID, rating in k_neighbours:
    try:
        similarities = similarity_matrix[itemID]
        for innerID, score in enumerate(similarities):
            candidates[innerID] += score * (rating / 5.0)
    except:
        continue

In [45]:
def getMovieName(movieID):
  if int(movieID) in tmp_data_movies:
    return tmp_data_movies[int(movieID)]
  else:
      return ""

In [46]:
watched = {}
for itemID, rating in trainset.ur[test_subject_iid]:
    watched[itemID] = 1

recommendations = []
position = 0
for itemID, rating_sum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
    if not itemID in watched:
        recommendations.append(getMovieName(trainset.to_raw_iid(itemID)))
        position += 1
        # only want top n which in our case in 10
        if(position > 10): break

for rec in recommendations:
    print("Movie: ", rec)


Movie:  Lord of the Rings: The Return of the King: Extended Edition: Bonus Material
Movie:  Mississippi Burning
Movie:  The Legend
Movie:  Firestarter
Movie:  Back to the Future Part III
Movie:  Fallen
Movie:  Richard Pryor: Live on the Sunset Strip
Movie:  George Carlin: Personal Favorites
Movie:  Christmas with The Simpsons
Movie:  Futurama: Monster Robot Maniac Fun Collection
Movie:  Unleashed


In [47]:
print("user", test_subject, 'previous recommendations')
display_recommendation(customer_id=test_subject, number_to_show=10, predictor=svd)

user 6 previous recommendations
Movies/TV Shows recommended to customer
                                                    estimated_score
movie_title                                                        
La Femme Nikita: Season 3                                  4.553385
Reno 911: Season 2                                         4.314836
Stevie Ray Vaughan and Double Trouble: Live at ...         4.306074
Baby Einstein: Baby Da Vinci                               4.235643
The Chorus                                                 4.232877
Ghosts of Rwanda: Frontline                                4.225248
Blue Planet: IMAX                                          4.220234
A Touch of Frost: Seasons 7 & 8                            4.219744
Foyle's War: Set 2                                         4.198104
Christmas with The Simpsons                                4.197200


In [48]:
#TODO:
#for the new user problem
## find highest rated movie / tv shows that have been watched the most -> how likely is new user to click on it ?